In [ ]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
import torch
from torch.utils.data import DataLoader

# Set up the transforms
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the dataset using ImageFolder
root_dir = '/content/drive/MyDrive/dataset/train'
dataset = ImageFolder(root=root_dir, transform=data_transforms)

# Split the dataset into training, validation, and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(dataset.imgs, dataset.targets, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Define the data loaders for each set
train_loader = DataLoader(dataset=torch.utils.data.Subset(dataset, indices=[i for i in range(len(X_train))]),
                          batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=torch.utils.data.Subset(dataset, indices=[i for i in range(len(X_val))]),
                          batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=torch.utils.data.Subset(dataset, indices=[i for i in range(len(X_test))]),
                          batch_size=32, shuffle=False)

# Print the sizes of the splits
print(f'Training set size: {len(X_train)}')
print(f'Validation set size: {len(X_val)}')
print(f'Testing set size: {len(X_test)}')

Training set size: 2700
Validation set size: 676
Testing set size: 844


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

# Set up the transforms
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the dataset using ImageFolder
root_dir = '/content/drive/MyDrive/dataset/train'
dataset = ImageFolder(root=root_dir, transform=data_transforms)

# Split the dataset into training, validation, and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(dataset.imgs, dataset.targets, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Define the data loaders for each set
train_loader = DataLoader(dataset=torch.utils.data.Subset(dataset, indices=[i for i in range(len(X_train))]),
                          batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=torch.utils.data.Subset(dataset, indices=[i for i in range(len(X_val))]),
                          batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=torch.utils.data.Subset(dataset, indices=[i for i in range(len(X_test))]),
                          batch_size=32, shuffle=False)

# Define the CNN model
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model and define the loss function and optimizer
model = MyCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test accuracy: {accuracy:.2f}%')

Test accuracy: 93.36%


In [ ]:
from PIL import Image

# Load an example image
image_path = '/content/drive/MyDrive/dataset/train/cataract/103_left.jpg'
image = Image.open(image_path)

# Apply the transforms used during training to preprocess the image
image = data_transforms(image)
image = image.unsqueeze(0)

# Pass the image through the model to obtain the predicted class
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output.data, 1)

# Print the predicted class
class_names = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']
print(f'Predicted class: {class_names[predicted.item()]}')

Predicted class: cataract
